# Special Kills Rating

In this notebook, we look at "special kills" that players earned in each round. Examples of special kills are double kills, triple kills, or clutches such as 1v2, 1v3s, etc.

By training a logistic regression model on special kills data during tournaments leading up to the championship torunament, we assigne each team a "Special kills rating". For each game leading up to the championship, we use logistic regression to predict the game outcome. Logistic regression assigns a probability that a given team will win a given game. A team's special kills rating is the average probability assigned to that team over all games leading up to the championships. We use this special kills rating as a feature in our final model to predict the championship tournament outcome.

In [144]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import seaborn as sns
sns.set_style("whitegrid")

from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [145]:
# #### Allows for scrolling through data
#pd.set_option("display.max_rows", None)

In [146]:
scores = pd.read_csv("../data/vct_2023/matches/scores.csv")
scores.head()

,Tournament,Stage,Match Type,Match Name,Team A,Team B,Team A Score,Team B Score,Match Result
0,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Team Liquid,Natus Vincere,0,2,Natus Vincere won
1,Valorant Champions 2023,Group Stage,Opening (D),DRX vs LOUD,DRX,LOUD,2,1,DRX won
2,Valorant Champions 2023,Group Stage,Opening (B),FUT Esports vs T1,FUT Esports,T1,2,0,FUT Esports won
3,Valorant Champions 2023,Group Stage,Opening (B),Evil Geniuses vs FunPlus Phoenix,Evil Geniuses,FunPlus Phoenix,2,0,Evil Geniuses won
4,Valorant Champions 2023,Group Stage,Winner's (D),Natus Vincere vs DRX,Natus Vincere,DRX,1,2,DRX won


In [147]:
maps_scores = pd.read_csv("../data/vct_2023/matches/maps_scores.csv")

In [148]:
maps_scores["Match Result"] = np.where(maps_scores["Team A Score"] > maps_scores["Team B Score"], maps_scores["Team A"], maps_scores["Team B"])
maps_scores.head()

,Tournament,Stage,Match Type,Match Name,Map,Team A,Team A Score,Team A Attacker Score,Team A Defender Score,Team A Overtime Score,Team B,Team B Score,Team B Attacker Score,Team B Defender Score,Team B Overtime Score,Duration,Match Result
0,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Team Liquid,11,6,5,NaN,Natus Vincere,13,7,6,NaN,1:18:55,Natus Vincere
1,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Bind,Team Liquid,15,7,5,3.0,Natus Vincere,17,7,5,5.0,1:22:57,Natus Vincere
2,Valorant Champions 2023,Group Stage,Opening (D),DRX vs LOUD,Lotus,DRX,13,7,5,1.0,LOUD,15,7,5,3.0,1:17:19,LOUD
3,Valorant Champions 2023,Group Stage,Opening (D),DRX vs LOUD,Split,DRX,13,8,5,NaN,LOUD,6,2,4,NaN,47:47,DRX
4,Valorant Champions 2023,Group Stage,Opening (D),DRX vs LOUD,Ascent,DRX,13,8,5,NaN,LOUD,8,4,4,NaN,NaN,DRX


The following datframe contains special kills data for each round of each game in the season.

In [149]:
rounds_kills = pd.read_csv("../data/vct_2023/matches/rounds_kills.csv")
rounds_kills.head()

,Tournament,Stage,Match Type,Match Name,Map,Round Number,Eliminator Team,Eliminator,Eliminator Agent,Eliminated Team,Eliminated,Eliminated Agent,Kill Type
0,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Round 1,Team Liquid,soulcas,astra,Natus Vincere,cNed,killjoy,2k
1,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Round 1,Team Liquid,soulcas,astra,Natus Vincere,Shao,fade,2k
2,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Round 17,Team Liquid,soulcas,astra,Natus Vincere,ANGE1,omen,2k
3,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Round 17,Team Liquid,soulcas,astra,Natus Vincere,Zyppan,raze,2k
4,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,Round 17,Team Liquid,soulcas,astra,Team Liquid,soulcas,astra,2k


Below, we merge the special kills data with the game outcome data.

In [150]:
left = rounds_kills
right = maps_scores[["Tournament", "Stage", "Match Type", "Match Name", "Map", "Match Result"]]
keys = ['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map']
rounds_kills_vs_outcome = pd.merge(left, right, on = keys)
rounds_kills_vs_outcome.shape

(70078, 14)

In [151]:
rounds_kills_vs_outcome["Won Match"] = np.where(rounds_kills_vs_outcome["Eliminator Team"] == rounds_kills_vs_outcome["Match Result"], "Win", "Loss")
rounds_kills_vs_outcome["Round Number"] = rounds_kills_vs_outcome['Round Number'].apply(lambda x: int(x[6:]))

In [152]:
rounds_kills_vs_outcome.sort_values(by=['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Round Number'], inplace=True)
rounds_kills_vs_outcome.head()

,Tournament,Stage,Match Type,Match Name,Map,Round Number,Eliminator Team,Eliminator,Eliminator Agent,Eliminated Team,Eliminated,Eliminated Agent,Kill Type,Match Result,Won Match
10002,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Ascent,1,Leviatán,Tacolilla,jett,KRÜ Esports,Klaus,sova,2k,KRÜ Esports,Loss
10003,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Ascent,1,Leviatán,Tacolilla,jett,KRÜ Esports,DaveeyS,kayo,2k,KRÜ Esports,Loss
10004,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Ascent,2,Leviatán,Tacolilla,jett,KRÜ Esports,Klaus,sova,2k,KRÜ Esports,Loss
10005,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Ascent,2,Leviatán,Tacolilla,jett,KRÜ Esports,keznit,jett,2k,KRÜ Esports,Loss
9976,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Ascent,3,KRÜ Esports,Klaus,sova,Leviatán,nzr,sova,2k,KRÜ Esports,Win


In [153]:
eco_rounds = pd.read_csv("../data/vct_2023/matches/eco_rounds.csv")
eco_rounds.rename(columns={'Outcome':'Won Round'}, inplace=True)
eco_rounds.rename(columns={'Team':'Eliminator Team'}, inplace=True)
eco_rounds.head()

,Tournament,Stage,Match Type,Match Name,Map,Round Number,Eliminator Team,Loadout Value,Remaining Credits,Type,Won Round
0,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,1,Team Liquid,3.6k,0.4k,Eco: 0-5k,Win
1,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,1,Natus Vincere,3.5k,0.3k,Eco: 0-5k,Loss
2,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,2,Team Liquid,15.6k,4.1k,Semi-buy: 10-20k,Win
3,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,2,Natus Vincere,4.2k,6.4k,Eco: 0-5k,Loss
4,Valorant Champions 2023,Group Stage,Opening (D),Team Liquid vs Natus Vincere,Fracture,3,Team Liquid,13.3k,14.8k,Semi-buy: 10-20k,Win


The eco_rounds dataframe contains information on the outcomes of individual rounds, so we will merge that data with our special kills data.

In [154]:
eco_rounds[["Tournament","Stage", "Match Type", "Match Name", "Map"]].drop_duplicates().shape

(771, 5)

In [155]:
left = rounds_kills_vs_outcome
right = eco_rounds[["Tournament", "Stage", "Match Type", "Match Name", "Map", "Round Number", "Eliminator Team", "Won Round"]]
keys = ['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Eliminator Team', 'Round Number']
rounds_kills_vs_outcome = left.merge(right, left_on=keys, right_on=keys)

In [156]:
team_rounds_kills = rounds_kills_vs_outcome.groupby(["Tournament", "Stage", "Match Type","Match Name", "Eliminator Team", "Map", "Round Number", "Won Round", "Won Match"])[['Eliminator']].count().reset_index()

In [157]:
team_rounds_kills = team_rounds_kills.rename(columns={"Eliminator":"Total_Special_Kills"})
team_rounds_kills = team_rounds_kills.rename(columns={"Round Number":"Round_Number"})
rounds_kills_vs_outcome = rounds_kills_vs_outcome.rename(columns={"Round Number":"Round_Number"})

In [158]:
rounds_kills_vs_outcome.sort_values(by=['Round_Number'])
rounds_kills_vs_outcome.head()

,Tournament,Stage,Match Type,Match Name,Map,Round_Number,Eliminator Team,Eliminator,Eliminator Agent,Eliminated Team,Eliminated,Eliminated Agent,Kill Type,Match Result,Won Match,Won Round
0,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Ascent,1,Leviatán,Tacolilla,jett,KRÜ Esports,Klaus,sova,2k,KRÜ Esports,Loss,Win
1,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Ascent,1,Leviatán,Tacolilla,jett,KRÜ Esports,DaveeyS,kayo,2k,KRÜ Esports,Loss,Win
2,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Ascent,2,Leviatán,Tacolilla,jett,KRÜ Esports,Klaus,sova,2k,KRÜ Esports,Loss,Win
3,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Ascent,2,Leviatán,Tacolilla,jett,KRÜ Esports,keznit,jett,2k,KRÜ Esports,Loss,Win
4,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Ascent,3,KRÜ Esports,Klaus,sova,Leviatán,nzr,sova,2k,KRÜ Esports,Win,Win


In [159]:
team_rounds_kills.head()

,Tournament,Stage,Match Type,Match Name,Eliminator Team,Map,Round_Number,Won Round,Won Match,Total_Special_Kills
0,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,3,Win,Win,4
1,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,4,Win,Win,2
2,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,5,Loss,Win,2
3,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,6,Loss,Win,3
4,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,7,Win,Win,3


### Special Kills Rating

Below we compute our special kills rating. First, we make a dataframe that contains the total number of special kills a team achieved in each round. For rounds in which no special kills occured, we assign a value of 0.

In [160]:
rounds = rounds_kills_vs_outcome[["Tournament", "Stage", "Match Type", "Match Name", "Eliminator Team", "Map","Won Match"]].drop_duplicates()
rounds.head()

,Tournament,Stage,Match Type,Match Name,Eliminator Team,Map,Won Match
0,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Leviatán,Ascent,Loss
4,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,Win
81,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Leviatán,Lotus,Loss
83,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Lotus,Win
158,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Leviatán,Pearl,Win


In [161]:
rounds = rounds.assign(Round_Number = [[i for i in range(1,eco_rounds["Round Number"].max())]]*len(rounds)).explode("Round_Number")

In [162]:
rounds.reset_index(drop = True, inplace = True)
rounds.head()

,Tournament,Stage,Match Type,Match Name,Eliminator Team,Map,Won Match,Round_Number
0,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Leviatán,Ascent,Loss,1
1,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Leviatán,Ascent,Loss,2
2,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Leviatán,Ascent,Loss,3
3,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Leviatán,Ascent,Loss,4
4,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Leviatán,Ascent,Loss,5


In [163]:
special_kills_all_rounds = pd.merge(team_rounds_kills, rounds, how='outer', on=["Tournament", "Stage", "Match Type", "Match Name", "Eliminator Team", "Map", "Round_Number", "Won Match"])

In [164]:
special_kills_all_rounds["Total_Special_Kills"] = special_kills_all_rounds["Total_Special_Kills"].fillna(0)
special_kills_all_rounds = special_kills_all_rounds.drop(columns = "Won Round");
special_kills_all_rounds.head()

,Tournament,Stage,Match Type,Match Name,Eliminator Team,Map,Round_Number,Won Match,Total_Special_Kills
0,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,1,Win,0.0
1,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,2,Win,0.0
2,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,3,Win,4.0
3,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,4,Win,2.0
4,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,5,Win,2.0


In [165]:
eco_rounds.rename(columns={'Round Number':'Round_Number'}, inplace=True)
special_kills_all_rounds = pd.merge(special_kills_all_rounds, eco_rounds[["Tournament", "Stage", "Match Type", "Match Name", "Eliminator Team", "Map", "Round_Number", "Won Round"]], how = 'outer', on=["Tournament", "Stage", "Match Type", "Match Name", "Eliminator Team", "Map", "Round_Number"])

In [166]:
special_kills_all_rounds = special_kills_all_rounds[["Tournament", "Stage", "Match Type", "Match Name", "Eliminator Team", "Map", "Round_Number", "Won Round", "Total_Special_Kills", "Won Match"]]
special_kills_all_rounds.head()

,Tournament,Stage,Match Type,Match Name,Eliminator Team,Map,Round_Number,Won Round,Total_Special_Kills,Won Match
0,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,1,Loss,0.0,Win
1,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,2,Loss,0.0,Win
2,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,3,Win,4.0,Win
3,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,4,Win,2.0,Win
4,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,5,Loss,2.0,Win


In [167]:
special_kills_all_rounds.shape

(60142, 10)

In [168]:
special_kills_all_rounds = special_kills_all_rounds.dropna()
special_kills_all_rounds.head()

,Tournament,Stage,Match Type,Match Name,Eliminator Team,Map,Round_Number,Won Round,Total_Special_Kills,Won Match
0,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,1,Loss,0.0,Win
1,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,2,Loss,0.0,Win
2,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,3,Win,4.0,Win
3,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,4,Win,2.0,Win
4,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,5,Loss,2.0,Win


In [169]:
keys = ['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map','Eliminator Team', "Won Match"]

special_kills_all_rounds[special_kills_all_rounds["Round_Number"] == 1].drop(columns=["Round_Number", "Won Round", "Total_Special_Kills"]);

In [170]:
df = special_kills_all_rounds[special_kills_all_rounds["Round_Number"] == 1].drop(columns=["Round_Number", "Total_Special_Kills","Won Round"])
df.head()

,Tournament,Stage,Match Type,Match Name,Eliminator Team,Map,Won Match
0,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,Win
39,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Lotus,Win
78,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Pearl,Loss
117,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Split,Win
156,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Leviatán,Ascent,Loss


We convert each round's special kills data into a column containing the number of special kills in that round for each team in each game.

In [171]:
for i in range(13):
    df = pd.merge(df,special_kills_all_rounds[special_kills_all_rounds["Round_Number"] == i + 1]\
            .drop(columns=["Round_Number", "Won Round"]), on = keys, how = "outer")\
            .rename(columns={"Total_Special_Kills":f"R{i+1}_Special_Kills"})

df.head()

,Tournament,Stage,Match Type,Match Name,Eliminator Team,Map,Won Match,R1_Special_Kills,R2_Special_Kills,R3_Special_Kills,R4_Special_Kills,R5_Special_Kills,R6_Special_Kills,R7_Special_Kills,R8_Special_Kills,R9_Special_Kills,R10_Special_Kills,R11_Special_Kills,R12_Special_Kills,R13_Special_Kills
0,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,Win,0.0,0.0,4.0,2.0,2.0,3.0,3.0,4.0,3.0,5.0,2.0,0.0,4.0
1,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Leviatán,Ascent,Loss,2.0,2.0,0.0,0.0,5.0,3.0,2.0,2.0,0.0,0.0,2.0,4.0,3.0
2,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Lotus,Win,0.0,2.0,3.0,2.0,2.0,4.0,3.0,2.0,3.0,0.0,0.0,2.0,5.0
3,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Leviatán,Lotus,Loss,2.0,4.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,3.0,4.0,2.0
4,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Pearl,Loss,0.0,0.0,2.0,3.0,2.0,4.0,2.0,2.0,2.0,0.0,5.0,2.0,4.0


We look at only tournaments leading up to the final champions tournament.

In [172]:
df_non_finals = df[df["Tournament"] != "Valorant Champions 2023"]

In [173]:
df_non_finals

,Tournament,Stage,Match Type,Match Name,Eliminator Team,Map,Won Match,R1_Special_Kills,R2_Special_Kills,R3_Special_Kills,R4_Special_Kills,R5_Special_Kills,R6_Special_Kills,R7_Special_Kills,R8_Special_Kills,R9_Special_Kills,R10_Special_Kills,R11_Special_Kills,R12_Special_Kills,R13_Special_Kills
0,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,Win,0.0,0.0,4.0,2.0,2.0,3.0,3.0,4.0,3.0,5.0,2.0,0.0,4.0
1,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Leviatán,Ascent,Loss,2.0,2.0,0.0,0.0,5.0,3.0,2.0,2.0,0.0,0.0,2.0,4.0,3.0
2,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Lotus,Win,0.0,2.0,3.0,2.0,2.0,4.0,3.0,2.0,3.0,0.0,0.0,2.0,5.0
3,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Leviatán,Lotus,Loss,2.0,4.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,3.0,4.0,2.0
4,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Pearl,Loss,0.0,0.0,2.0,3.0,2.0,4.0,2.0,2.0,2.0,0.0,5.0,2.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1371,Champions Tour 2023: Pacific League,Playoffs,Upper Semifinals,DRX vs Team Secret,Team Secret,Split,Loss,0.0,2.0,0.0,5.0,2.0,0.0,0.0,4.0,2.0,0.0,0.0,3.0,2.0
1372,Champions Tour 2023: Pacific League,Playoffs,Upper Semifinals,Paper Rex vs T1,Paper Rex,Fracture,Win,2.0,0.0,4.0,0.0,2.0,4.0,4.0,3.0,3.0,3.0,3.0,2.0,2.0
1373,Champions Tour 2023: Pacific League,Playoffs,Upper Semifinals,Paper Rex vs T1,T1,Fracture,Loss,0.0,4.0,2.0,2.0,3.0,0.0,5.0,0.0,3.0,0.0,2.0,0.0,2.0
1374,Champions Tour 2023: Pacific League,Playoffs,Upper Semifinals,Paper Rex vs T1,Paper Rex,Lotus,Win,4.0,2.0,0.0,0.0,5.0,0.0,0.0,2.0,4.0,5.0,4.0,2.0,2.0


In [174]:
df.Tournament.unique()

array(['Champions Tour 2023: Americas Last Chance Qualifier',
       'Champions Tour 2023: Americas League',
       'Champions Tour 2023: Champions China Qualifier',
       'Champions Tour 2023: EMEA Last Chance Qualifier',
       'Champions Tour 2023: EMEA League',
       'Champions Tour 2023: Lock-In Sao Paulo',
       'Champions Tour 2023: Masters Tokyo',
       'Champions Tour 2023: Pacific Last Chance Qualifier',
       'Champions Tour 2023: Pacific League', 'Valorant Champions 2023'],
      dtype=object)

In [175]:
df_non_finals.keys()

Index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Eliminator Team',
       'Map', 'Won Match', 'R1_Special_Kills', 'R2_Special_Kills',
       'R3_Special_Kills', 'R4_Special_Kills', 'R5_Special_Kills',
       'R6_Special_Kills', 'R7_Special_Kills', 'R8_Special_Kills',
       'R9_Special_Kills', 'R10_Special_Kills', 'R11_Special_Kills',
       'R12_Special_Kills', 'R13_Special_Kills'],
      dtype='object')

Now we use logistic regression on special data in pre-finals tournaments to assign our special kill score.

In [176]:
X = df_non_finals[['R1_Special_Kills', 'R2_Special_Kills',\
       'R3_Special_Kills', 'R4_Special_Kills', 'R5_Special_Kills',\
       'R6_Special_Kills', 'R7_Special_Kills', 'R8_Special_Kills',\
       'R9_Special_Kills', 'R10_Special_Kills', 'R11_Special_Kills',\
       'R12_Special_Kills', 'R13_Special_Kills']]
X = X.fillna(0)
y = df_non_finals["Won Match"] == "Win"

In [177]:
lr_kills_rating_non_finals = LogisticRegression()
lr_kills_rating_non_finals.fit(X,y)
pred = lr_kills_rating_non_finals.predict(X)
acc = accuracy_score(y, pred)

In [178]:
print(acc)

0.7296511627906976


In [179]:
special_kills_rating_non_finals = lr_kills_rating_non_finals.predict_proba(X)[:,1]

In [180]:
special_kills_non_finals = df_non_finals[['Tournament', 'Stage', 'Match Type', 'Match Name', 'Eliminator Team', 'Map', 'Won Match']]
special_kills_non_finals.loc[:,"SK_Rating"] = special_kills_rating_non_finals

C:\Users\alexe\AppData\Local\Temp\ipykernel_12024\92566189.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  special_kills_non_finals.loc[:,"SK_Rating"] = special_kills_rating_non_finals


We append our special kills rating to our dataframe and export as a pdf for use in our final analysis.

In [181]:
special_kills_non_finals.head()

,Tournament,Stage,Match Type,Match Name,Eliminator Team,Map,Won Match,SK_Rating
0,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Ascent,Win,0.672598
1,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Leviatán,Ascent,Loss,0.484018
2,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Lotus,Win,0.541446
3,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,Leviatán,Lotus,Loss,0.349981
4,Champions Tour 2023: Americas Last Chance Qual...,Main Event,Grand Final,KRÜ Esports vs Leviatán,KRÜ Esports,Pearl,Loss,0.524159


In [182]:
# Save a dataframe containing a column with the special kills rating
special_kills_non_finals.to_csv("../data/vct_2023/special_kills_non_finals.csv")

In [183]:
# Save a dataframe containing all a column of special kills for each of the first 13 rounds
df.to_csv("../data/vct_2023/special_kills_non_finals_rounds.csv")